## TD 2 distance

In [1]:
import pandas as pd
import glob
import os
import numpy as np
import sys
! pip install pmcx
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import os
import pickle
import numpy as np
from pathlib import Path
import os, re

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 45.9 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')
folder_path = '/content/drive/MyDrive/MCX_data'
sys.path.append(folder_path)
from FD_msNIRS_2layers import extract_freq as ef

Mounted at /content/drive


### GT

In [3]:
train_sorted_ids = [i+1 for i in range(10000)]
GT_folder_train = '/content/drive/MyDrive/MCX_data/stage2_csv/'
csv_files = glob.glob(os.path.join(GT_folder_train, '*.csv'))
GT_all_train = pd.concat([pd.read_csv(f) for f in csv_files], ignore_index=True)

# Ensure ID column is integer
GT_all_train['ID'] = GT_all_train['ID'].astype(int)

# Filter and sort GT_all to match the sorted_ids order
GT_filtered = GT_all_train[GT_all_train['ID'].isin(train_sorted_ids)].copy()
GT_filtered = GT_filtered.set_index('ID').loc[train_sorted_ids].reset_index()

# Extract HHB2 as Y
Y_train = GT_filtered['HHB2'].values  # numpy array (n_samples,)
Y_train = Y_train.reshape(-1, 1)
print(Y_train.shape)

(10000, 1)


In [4]:
test_sorted_ids = [i+1 for i in range(1000)]
GT_folder_test = '/content/drive/MyDrive/MCX_data/test_csv/'
csv_files = glob.glob(os.path.join(GT_folder_test, '*.csv'))
GT_all_test = pd.concat([pd.read_csv(f) for f in csv_files], ignore_index=True)

GT_all_test['ID'] = GT_all_test['ID'].astype(int)

GT_filtered = GT_all_test[GT_all_test['ID'].isin(test_sorted_ids)].copy()
GT_filtered = GT_filtered.set_index('ID').loc[test_sorted_ids].reset_index()

Y_test = GT_filtered['HHB2'].values  # numpy array (n_samples,)
Y_test = Y_test.reshape(-1, 1)
print(Y_test.shape)

(1000, 1)


#### scale X and Y

In [5]:
from sklearn.preprocessing import StandardScaler
import joblib

scaler_y  = StandardScaler()
Y_train = scaler_y.fit_transform(Y_train.reshape(-1, 1)).ravel()
Y_test  = scaler_y.transform(Y_test.reshape(-1, 1)).ravel()

In [6]:
pth = '/content/drive/MyDrive/MCX_data/weights_td/'
joblib.dump(scaler_y, pth+"TD_thickness_CNN_scaler_y.pkl")
SAVE_DIR = Path("/content/drive/MyDrive/MCX_data/splits")
data    = np.load(SAVE_DIR / "train_test_split.npz", allow_pickle=False)
X_train = data["X_train"]
X_test  = data["X_test"]

In [7]:
print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)

(10000, 64, 1000) (1000, 64, 1000) (10000,) (1000,)


### Create Seperations

In [8]:
import numpy as np
import itertools
from typing import Dict, List

import numpy as np
from typing import Dict, List

def split_distance_prefix(
    X_train: np.ndarray,   # (N_train, 64, 1000)
    X_test:  np.ndarray,   # (N_test,   64, 1000)
    min_len: int = 3,
    max_len: int = 7
) -> Dict[int, List[np.ndarray]]:
    """
    Build a dict of prefix combinations of distances 1…L for L in [min_len…max_len].
    Keys are integers like 123 (for distances 1–3), 1234 (1–4), …, up to 1234567 (1–7).
    Each value is [X_train_slice, X_test_slice] with shape (N, 8*L, 1000).

    Parameters
    ----------
    X_train : np.ndarray
        Training data of shape (N_train, 64, 1000) where 64 = 8 distances × 8 wavelengths.
    X_test : np.ndarray
        Test data of shape (N_test, 64, 1000).
    min_len : int, default=3
        Minimum prefix length (e.g. 3 → distances 1,2,3).
    max_len : int, default=7
        Maximum prefix length (e.g. 7 → distances 1…7).

    Returns
    -------
    prefix_dict : Dict[int, List[np.ndarray]]
        Dictionary with keys {123, 1234, 12345, 123456, 1234567} mapping to
        [X_train_prefix, X_test_prefix].
    """
    # Validate input dimensions
    if X_train.ndim != 3 or X_test.ndim != 3 or X_train.shape[1] != 64 or X_test.shape[1] != 64:
        raise ValueError("Input arrays must have shape (N, 64, 1000).")

    prefix_dict: Dict[int, List[np.ndarray]] = {}

    # Build prefixes of lengths from min_len to max_len
    for L in range(min_len, max_len + 1):
        # distances 1 through L
        distances = list(range(1, L + 1))
        key = int("".join(map(str, distances)))  # e.g. [1,2,3] -> 123

        # slice out each block of 8 channels per distance
        train_blocks = [X_train[:, (d-1)*8 : d*8, :] for d in distances]
        test_blocks  = [X_test[:,  (d-1)*8 : d*8, :] for d in distances]

        # concatenate along the channel axis
        Xtr_pref = np.concatenate(train_blocks, axis=1)  # shape (N_train, 8*L, 1000)
        Xte_pref = np.concatenate(test_blocks,  axis=1)  # shape (N_test,   8*L, 1000)

        prefix_dict[key] = [Xtr_pref, Xte_pref]

    return prefix_dict

distance_data = split_distance_prefix(X_train, X_test)

In [9]:
distance_data.keys()

dict_keys([123, 1234, 12345, 123456, 1234567])

In [10]:
len(distance_data)

5

### ANN

In [19]:
from tensorflow.keras.layers import (
    Input, Conv2D, Activation, Dropout, Add,
    MaxPooling2D, GlobalAveragePooling2D, Dense
)
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow as tf

def residual_block(x, filters, kernel_size=(5, 5), downsample=False, dropout_rate=0.0):
    """A single residual block with optional downsampling."""
    shortcut = x

    # First conv
    x = Conv2D(filters, kernel_size,
               padding='same',
               strides=(2, 2) if downsample else (1, 1)
    )(x)
    x = Activation('relu')(x)
    x = Dropout(dropout_rate)(x)

    # Second conv
    x = Conv2D(filters, kernel_size, padding='same')(x)

    # Shortcut path
    if downsample or shortcut.shape[-1] != filters:
        shortcut = Conv2D(
            filters, (1, 1),
            strides=(2, 2) if downsample else (1, 1),
            padding='same'
        )(shortcut)

    # Merge & final activation
    x = Add()([x, shortcut])
    x = Activation('relu')(x)
    return x

def build_crompresnet_model(input_tensor, dropout_rate=0.0) -> Model:
    """
    Builds the ResNet-like model as described:
      • Input assumed shape=(16, 1000, 1)
      • Initial conv + max‑pool
      • 3 groups of residual blocks (128→128, then 256→256)
      • Global average pooling + dense head
    """
    # Initial conv & pool
    x = Conv2D(32, (3, 3), padding='same', activation='relu')(input_tensor)
    x = MaxPooling2D(pool_size=(2, 2))(x)

    # Residual groups
    x = residual_block(x, 128, downsample=True,  dropout_rate=dropout_rate)
    x = residual_block(x, 128, downsample=False, dropout_rate=dropout_rate)

    x = residual_block(x, 128, downsample=True,  dropout_rate=dropout_rate)
    x = residual_block(x, 128, downsample=False, dropout_rate=dropout_rate)

    x = residual_block(x, 256, downsample=True,  dropout_rate=dropout_rate)
    x = residual_block(x, 256, downsample=False, dropout_rate=dropout_rate)

    # Head
    x = GlobalAveragePooling2D()(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(dropout_rate)(x)
    x = Dense(64,  activation='relu')(x)
    output = Dense(1)(x)

    # Wrap into a Model
    return Model(inputs=input_tensor, outputs=output)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
Path = '/content/drive/MyDrive/MCX_data/distance_all/'
for k, v in distance_data.items():
     input_tensor = Input(shape=(int(8*len(str(k))), 1000, 1))
     print(input_tensor.shape)
     resnet_model = build_crompresnet_model(input_tensor, dropout_rate=0.0)
     checkpoint_cb = ModelCheckpoint(
        "TD_hhb2_CNN.keras",        # filepath to save to
        monitor="val_loss",     # quantity to monitor
        mode="min",             # we want to minimize val_loss
        save_best_only=True,    # only save when val_loss decreases
        save_weights_only=False # set True if you only want weights
     )
     optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
     resnet_model.compile(optimizer=optimizer, loss="mse")
     history = resnet_model.fit( v[0], Y_train, epochs=200, batch_size=32, validation_split=0.1, verbose=1, callbacks=[checkpoint_cb])
     resnet_model = tf.keras.models.load_model("TD_hhb2_CNN.keras")
     y_pred = resnet_model.predict(v[1])
     y_pred = np.clip(y_pred, scaler_y.transform(np.array([[15]]))[0][0], scaler_y.transform(np.array([[35]]))[0][0])
     ###
     y_pred = scaler_y.inverse_transform(y_pred.reshape(-1, 1)).flatten()
     y_test = scaler_y.inverse_transform(Y_test.reshape(-1, 1)).flatten()
     mse = mean_squared_error(y_test, y_pred)
     print(f"MSE for distance {k}: {mse}")
     print(f"RMSE for distance {k}: {np.sqrt(mse)}")
     df = pd.DataFrame({
        "y_test" : y_test,
        "y_pred" : y_pred
    })
     df.to_csv(Path + f"distance_{k}_hhb2_TD.csv", index=False)

(None, 24, 1000, 1)
Epoch 1/200
282/282 ━━━━━━━━━━━━━━━━━━━━ 19s 38ms/step - loss: 0.9167 - val_loss: 0.5747
Epoch 2/200
282/282 ━━━━━━━━━━━━━━━━━━━━ 5s 18ms/step - loss: 0.5099 - val_loss: 0.3601
Epoch 3/200
282/282 ━━━━━━━━━━━━━━━━━━━━ 4s 16ms/step - loss: 0.3830 - val_loss: 0.3724
Epoch 4/200
282/282 ━━━━━━━━━━━━━━━━━━━━ 5s 18ms/step - loss: 0.3487 - val_loss: 0.2994
Epoch 5/200
282/282 ━━━━━━━━━━━━━━━━━━━━ 5s 18ms/step - loss: 0.2760 - val_loss: 0.2812
Epoch 6/200
282/282 ━━━━━━━━━━━━━━━━━━━━ 5s 18ms/step - loss: 0.2382 - val_loss: 0.2700
Epoch 7/200
282/282 ━━━━━━━━━━━━━━━━━━━━ 5s 17ms/step - loss: 0.2125 - val_loss: 0.1990
Epoch 8/200
282/282 ━━━━━━━━━━━━━━━━━━━━ 5s 18ms/step - loss: 0.1805 - val_loss: 0.1884
Epoch 9/200
282/282 ━━━━━━━━━━━━━━━━━━━━ 4s 16ms/step - loss: 0.1564 - val_loss: 0.2589
Epoch 10/200
282/282 ━━━━━━━━━━━━━━━━━━━━ 5s 18ms/step - loss: 0.1818 - val_loss: 0.1793
Epoch 11/200
282/282 ━━━━━━━━━━━━━━━━━━━━ 4s 16ms/step - loss: 0.1407 - val_loss: 0.1846
Epoch 12/